In [ ]:
#Python File for ETL Project

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
#Extract CSV's into DataFrames

In [ ]:
# File Legend:

# CSV_1: Netflix Titles
# CSV_2: Metascore Data 


In [ ]:
# Extract Step
# Netflix Titles

CSV_1 = "Resources/netflix_titles.csv"
netflix_df = pd.read_csv(CSV_1)
netflix_df.head()

In [ ]:
# List total columns & Identify columns for revised dataframe

for col in netflix_df.columns: 
    print(col) 
    

In [ ]:
# Transform Step
# Revise Dataset with select columns

revised_netflix_df = netflix_df[["type","title","director","country","release_year","listed_in","description"]]
revised_netflix_df.head()


In [ ]:
# Transform Step
# Drop TV Type 
drop_netflix_df = revised_netflix_df[revised_netflix_df.type != 'TV Show']
drop_netflix_df.head()

In [ ]:
# Transform Step
# Rename columns
renamed_netflix_df = drop_netflix_df.rename(columns={"title":"movie_title","country":"produced_in","listed_in": "netflix_genre"})
renamed_netflix_df.head()

In [ ]:
# Transform Step
# Remove Type Column
movie_netflix_df = renamed_netflix_df.drop('type', axis=1)
movie_netflix_df.head()

In [ ]:
# Transform Step 
# split produced_in
# Define 1st value in listed_in column as main country the movie was produced in
produced_df = movie_netflix_df

# split genre column
split_produced_df = produced_df['produced_in'].str.split(',', 1, expand=True)

# Rename columns
renamed_produced_df = split_produced_df.rename(columns={0:"primary_country",1:"other_countries"})
renamed_produced_df.head()

# drop column
country_df = renamed_produced_df.drop('other_countries', axis=1)
country_df.head()


In [ ]:
# Get Unique Values in 
country_list = country_df['primary_country'].unique()
country_list

In [ ]:
# Classify International Movies 
# might need to be done by merging country_df with movie_netflix_df and then grouping US films... 


In [ ]:
# next steps: - JP Can do 
# filter type to just movies 
# remove description column
# rename listed_in to 'Netflix Genre': extract first listed_in value as Netflix Genre
# could filter out international movies, country to United States 

In [ ]:
# split neftlix_genre
# Define 1st value in listed_in column as main country the movie was produced in
genre_df = movie_netflix_df

# split genre column
split_genre_df = genre_df['netflix_genre'].str.split(',', 1, expand=True)

# Rename columns
renamed_genre_df = split_genre_df.rename(columns={0:"primary_genre",1:"other_genres"})

# drop column
drop_netflix_genre_df = renamed_genre_df.drop('other_genres', axis=1)

# Rename columns
netflix_genre_df = drop_netflix_genre_df.rename(columns={"primary_genre":"listed_in_netflix"})
netflix_genre_df.head()

In [ ]:
# Extract Step
# Metascore Data

CSV_2 = "Resources/metacritic_movies.csv"
metascore_df = pd.read_csv(CSV_2)
metascore_df.head()

In [ ]:
#List total columns & identify columns for revised dataframe
for col in metascore_df.columns: 
    print(col) 

In [ ]:
# Transform Step
# Revise Dataset with select columns

revised_metascore_df = metascore_df[["movie_title","genre","metascore"]]
revised_metascore_df.head()


In [ ]:
# Transform Step
# Rename columns
renamed_metascore_df = revised_metascore_df.rename(columns={"genre": "metacritic_genre"})
renamed_metascore_df.head()


In [ ]:
# split metacritic_genre
# Define 1st value in listed_in column as main country the movie was produced in
metagenre_df = renamed_metascore_df[['movie_title','metacritic_genre']]
metagenre_df.head()

In [ ]:
# split genre column
split_metagenre_df = renamed_metascore_df['metacritic_genre'].str.split(',', 1, expand=True)


In [ ]:

# Rename columns
renamed_metagenre_df = split_metagenre_df.rename(columns={0:"primary_genre",1:"other_genres"})

# drop column
drop_genre_df = renamed_metagenre_df.drop('other_genres', axis=1)

# Rename column
metacritic_genre_df = drop_genre_df.rename(columns={"primary_genre":"metacritic_genre"})
metacritic_genre_df.head()

In [ ]:
new_metacritic_df = pd.merge(renamed_metascore_df, metacritic_genre_df, on="metacritic_genre")
new_metacritic_df.head()

In [ ]:
# Binning by score range 
#score range legend (4 star range)

# 4-Star Scale
# Their Grade	Converts to
# 4- 100
# 3 -75
# 2 - 50
# 1 - 25
# 0 - 0

 # Create bins in which to place values based upon TED Talk views
bins = [0, 25, 50, 75, 100]

# Create labels for these bins
group_labels = ["1 star", "2 stars", "3 stars", "4 stars"]

# Place the data series into a new column inside of the DataFrame
renamed_metascore_df["Rating"] = pd.cut(renamed_metascore_df["metascore"], bins, labels=group_labels)
bin_df = renamed_metascore_df
bin_df.head()

In [ ]:
# Groupby
# Create a GroupBy object based upon "View Group"
new_bin_df = bin_df.groupby("Rating")
new_bin_df.head(2)

In [ ]:
# Sort Descending metascore
bin_df.sort_values(by='metascore', ascending=False)
bin_df.head()

In [ ]:
# attempt to merge netflix & metascore dfs

merge_movie_df = pd.merge(movie_netflix_df,bin_df, on='movie_title')
merge_movie_df.head()

In [ ]:
# Sort Descending metascore
sort_merge_df = merge_movie_df.sort_values(by='metascore', ascending=False)
use_this_df = sort_merge_df
use_this_df.head()

In [ ]:
#  Create database connection (need to update.. this is from class example)
# # connection_string = "postgres:postgres@localhost:5432/customer_db"
# engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
#  # Confirm tables
# engine.table_names()

In [ ]:
#  Load DataFrames into database (need to update.. this is from class example)

In [ ]:
#  premise_transformed.to_sql(name='premise', con=engine, if_exists='append', index=True)

In [ ]:
# county_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)